## Combining all downstream used A2TEA data into one RDS object
## Also performing necessary data manipulation

In [31]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

#BiocManager::install("DESeq2")


#lfcShrinkage isn't installable at the moment - problem for later
#install.packages("mvtnorm")
#BiocManager::install("apeglm")

Warning message in install.packages("mvtnorm"):
“installation of package ‘mvtnorm’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [52]:
library(DESeq2)
#library(apeglm)
library(BiocGenerics)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()      masks matrixStats::count()
✖ dplyr::desc()       masks IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::Position() masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()     masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()     masks S4Vectors::rename()
✖ purrr::simplify()   masks DelayedArray::simplify()
✖ dplyr::slice()      masks IRanges::slice()



#### Find all DESeq2 differential expression RDS objects and load them
##### Names are given based on the species/ecotype/etc. name

In [213]:
dea_list <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=TRUE)
dea_list_short <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=FALSE)

#species <- str_sub(dea_list_short[[1]], start=5)

lapply(dea_list_short, str_sub(start=5))


listRDS <- lapply(dea_list, readRDS, .GlobalEnv)


for (i in 1:length(listRDS)){
    assign(str_sub(dea_list_short[[i]], start=5), listRDS[[i]])
}

#remove unused combined list object
rm(listRDS)

In [20]:
#nice for later - but needs install of package "apeglm"

#link on logFC shrinking
#https://support.bioconductor.org/p/77461/
#?lfcShrink
#lfcShrink(arabidopsis_dea, coef=2, type="apeglm")
#resultsNames(arabidopsis_dea)

ERROR: Error in lfcShrink(arabidopsis_dea, coef = 2, type = "apeglm"): type='apeglm' requires installing the Bioconductor package 'apeglm'


In [273]:
species_list <- vector()

for (i in 1:length(dea_list_short)){
    species_list <- c(species_list, str_sub(dea_list_short[[i]], start=5))
}

list_AllSpeciesDEResultsDataFrames <- list()

for (i in species_list) {
    # individual could be deleted but perhaps I'll need them later; kept for now...
    assign(paste0(i, "_DEresultsTable"), as.data.frame(results(get(i))))
    # create gene name vector here (when list is constructed rownames become unique species-gene combination)
    gene <- rownames(get(paste0(i, "_DEresultsTable")))
    df_with_genes <- add_column(get(paste0(i, "_DEresultsTable")), gene, .before = "baseMean")
    # also add a species column which will will also come in handy during the shiny steps
    species <- replicate(nrow(df_with_genes), i)
    df_with_genes <- add_column(df_with_genes, species, .before = "gene")
    #create list of dataframes, which will come in handy
    list_AllSpeciesDEResultsDataFrames[[i]] <- df_with_genes
    #list_AllSpeciesDEResultsDataFrames[[i]] <- (get(paste0(i, "_DEresultsTable")))
}

#str(list_AllSpeciesDEResultsDataFrames)

#create combined "mega" dataframe of all species, which is going  to be used for shiny lookup

combined_AllSpeciesDEResultsDataFrames <- do.call("rbind", list_AllSpeciesDEResultsDataFrames)


#str(combined_AllSpeciesDEResultsDataFrames)
#head(combined_AllSpeciesDEResultsDataFrames)
#head(list_AllSpeciesDEResultsDataFrames[[1]])
#?rbind

'data.frame':	113939 obs. of  8 variables:
 $ species       : chr  "Arabidopsis_thaliana" "Arabidopsis_thaliana" "Arabidopsis_thaliana" "Arabidopsis_thaliana" ...
 $ gene          : chr  "AT1G01010" "AT1G01020" "AT1G01030" "AT1G01040" ...
 $ baseMean      : num  41.2 248.8 98.6 349.3 490.9 ...
 $ log2FoldChange: num  -0.1422 0.0483 0.1553 0.1579 -0.0106 ...
 $ lfcSE         : num  0.263 0.225 0.242 0.211 0.182 ...
 $ stat          : num  -0.5397 0.2148 0.643 0.749 -0.0584 ...
 $ pvalue        : num  0.589 0.83 0.52 0.454 0.953 ...
 $ padj          : num  1 1 1 1 1 ...


,species,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Arabidopsis_thaliana.AT1G01010,Arabidopsis_thaliana,AT1G01010,41.19547,-0.14217239,0.2634325,-0.53969189,0.5894095,0.9999255
Arabidopsis_thaliana.AT1G01020,Arabidopsis_thaliana,AT1G01020,248.82425,0.04827472,0.2247402,0.21480238,0.8299214,0.9999255
Arabidopsis_thaliana.AT1G01030,Arabidopsis_thaliana,AT1G01030,98.55767,0.15532807,0.2415669,0.64300226,0.5202226,0.9999255
Arabidopsis_thaliana.AT1G01040,Arabidopsis_thaliana,AT1G01040,349.28986,0.15787058,0.2107789,0.74898674,0.4538652,0.9999255
Arabidopsis_thaliana.AT1G01050,Arabidopsis_thaliana,AT1G01050,490.88704,-0.01060098,0.1815111,-0.05840401,0.9534268,0.9999255
Arabidopsis_thaliana.AT1G01060,Arabidopsis_thaliana,AT1G01060,504.75867,-0.18656451,0.2859922,-0.65234137,0.5141810,0.9999255


In [269]:
t <- replicate(5, "my_string")
t

[1] "my_string" "my_string" "my_string" "my_string" "my_string"

### creating long format HOG-genes relation table

In [283]:
HOG_file_raw <- read_delim("orthofinder/final-results/Phylogenetic_Hierarchical_Orthogroups/N0.tsv", delim = "\t")

#drop the OG, Gene Tree Parent Clade
HOG_file_short <- HOG_file_raw %>% select (-c(OG, `Gene Tree Parent Clade`)) 


#merge all species columns into one and remove the solo ones
#use all colnames except first - thus all species
HOG_file_merged <- HOG_file_short %>% unite(x,
                                          c(colnames(HOG_file_test)[-1]),
                                          sep = ",", 
                                          na.rm = TRUE,
                                          remove = TRUE)

#transform into long format - each row 1 gene and it's corresponding HOG in seperate columns
HOG_file_long <- HOG_file_merged %>%
                    mutate(unpacked = str_split(x, ",")) %>%
                    unnest(cols = c(unpacked)) %>%
                    mutate(genes = str_trim(unpacked)) %>% 
                    select(-c(x, unpacked)) %>% 
                    rename(gene = genes)

str(HOG_file_long)

Parsed with column specification:
cols(
  HOG = col_character(),
  OG = col_character(),
  `Gene Tree Parent Clade` = col_character(),
  Arabidopsis_thaliana = col_character(),
  Hordeum_vulgare = col_character(),
  Zea_mays = col_character()
)



tibble [90,454 × 2] (S3: tbl_df/tbl/data.frame)
 $ HOG : chr [1:90454] "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" ...
 $ gene: chr [1:90454] "Zm00001d020494" "Zm00001d007563" "Zm00001d052359" "Zm00001d037178" ...


### adding specific HOG or singleton info as HOG column to DE tables

In [284]:
#test <- as.data.frame(results(Arabidopsis_thaliana_dea))
#genes <- rownames(test)
#test2 <- add_column(test, genes, .before = "baseMean")
#test2


full_join(combined_AllSpeciesDEResultsDataFrames, HOG_file_long, 
          by = c("gene")#, 
#          suffix = c("_grass","_grain")
         )



species,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,HOG
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Arabidopsis_thaliana,AT1G01010,41.195470,-0.14217239,0.2634325,-0.53969189,0.5894095,0.9999255,N0.HOG0007194
Arabidopsis_thaliana,AT1G01020,248.824248,0.04827472,0.2247402,0.21480238,0.8299214,0.9999255,N0.HOG0005477
Arabidopsis_thaliana,AT1G01030,98.557671,0.15532807,0.2415669,0.64300226,0.5202226,0.9999255,N0.HOG0016327
Arabidopsis_thaliana,AT1G01040,349.289858,0.15787058,0.2107789,0.74898674,0.4538652,0.9999255,N0.HOG0011521
Arabidopsis_thaliana,AT1G01050,490.887038,-0.01060098,0.1815111,-0.05840401,0.9534268,0.9999255,N0.HOG0000568
Arabidopsis_thaliana,AT1G01060,504.758668,-0.18656451,0.2859922,-0.65234137,0.5141810,0.9999255,N0.HOG0008788
Arabidopsis_thaliana,AT1G01070,3.147454,-2.34442042,3.5749326,-0.65579429,0.5119565,0.9999255,N0.HOG0002270
Arabidopsis_thaliana,AT1G01080,1092.806741,-0.10177468,0.1428082,-0.71266696,0.4760519,0.9999255,N0.HOG0012587
Arabidopsis_thaliana,AT1G01090,2113.288680,-0.15603571,0.1387895,-1.12426162,0.2609021,0.9999255,N0.HOG0011778


### load in the rest of the data - hypotheses, trees, fasta and msa for the start 
### + general stats once Orthofinder calc. for HOG)